## Analyze azimuth scans for instrumental polarization
### A. Ordog, July 2022
### NOTE: please do not modify

In [ ]:
#import dva_sdhdf_combine
import imp
import os
import subprocess
import h5py
import numpy as np
from astropy.time import Time
import matplotlib.pyplot as plt
from scipy.interpolate import griddata
import datetime
import matplotlib.dates as mdates
from matplotlib.dates import HourLocator as HourLocator
from matplotlib.dates import MinuteLocator as MinuteLocator
from mpl_toolkits.axes_grid1 import make_axes_locatable
from astropy import units as u
from astropy.time import TimeDelta
from scipy import interpolate
from operator import itemgetter
from itertools import groupby

day ='27'
trim_az_exist = True

#### Change the directory to where the files are located" ####
dir_in_az = '/media/ordoga/DVA_data/survey_phase1_day'+day+'/'
#dir_out = '/media/ordoga/DVA_data/survey_phase1_day'+day+'/'
##############################################################

In [ ]:
scan_id = []    # The scan id number
scan_start = []  # Start time of the scan (UTC)
scan_stop = []   # Stop time of the scan (UTC)

# Read in the azimuth scan data and store it in arrays:
with open(dir_in_az+'DVAsurvey_phase1_day0'+day+'.txt') as fp:
    for line in fp:       
        scan_id.append(int(line.split()[0]))
        scan_start.append(line.split()[1]+'T'+line.split()[2][0:12])
        scan_stop.append(line.split()[3]+'T'+line.split()[4][0:12])

for i in range(0,len(scan_id)):
    print(f"{scan_id[i]:04}",scan_start[i],scan_stop[i])
print('')

# Convert start and stop times to Modified Julian Day (MJD).
scan_start_mjd = Time(scan_start, format='isot',scale='utc').mjd
scan_stop_mjd  = Time(scan_stop,  format='isot',scale='utc').mjd

In [ ]:
%%time

def concatenate_data(file,RR,LL,reRL,imRL,trim,dec,ra,el,az,t,noise,trim_exist):
    
    #print('')
    #print(trim_exist)
    #print('')
    
    dataset = file['data']['beam_0']['band_SB0']['scan_0']
    
    if trim_exist == True:
        trim_flag = dataset['metadata']['trim_scan_flag']
    else:
        trim_flag = np.zeros_like(dataset['metadata']['noise_state'])
    w = np.where(trim_flag == 0)[0]
    
    # Add the position and time data to the corresponding arrays:
    dec = np.concatenate([dec,dataset['metadata']['declination'][w]])
    ra = np.concatenate([ra,dataset['metadata']['right_ascension'][w]])
    el = np.concatenate([el,dataset['metadata']['elevation'][w]])
    az = np.concatenate([az,dataset['metadata']['azimuth'][w]])
    t = np.concatenate([t,dataset['metadata']['utc'][w]])
    noise = np.concatenate([noise,dataset['metadata']['noise_state'][w]])

    # Add the spectrometer power data to the corresponding arrays:
    RR = np.concatenate([RR,dataset['data'][w,0,:]],axis=0)
    LL = np.concatenate([LL,dataset['data'][w,1,:]],axis=0)
    reRL = np.concatenate([reRL,dataset['data'][w,2,:]],axis=0)
    imRL = np.concatenate([imRL,dataset['data'][w,3,:]],axis=0)
    
    return RR,LL,reRL,imRL,trim,dec,ra,el,az,t,noise

t = []
az = []
dec = []
ra = []
el = []
noise = []
trim_flag = []

# Use one of the scans to get the list of frequencies:
scan0 = f"{scan_id[0]:04}"
file = h5py.File(dir_in_az+'dva_survey_raw_scan_'+scan0+'.h5','r')
freq = file['data']['beam_0']['band_SB0']['frequency'][:]/1e6
df = freq[1]-freq[0]

# Create empty arrays for the power data:
RR = np.empty([0,len(freq)])
LL = np.empty([0,len(freq)])
reRL = np.empty([0,len(freq)])
imRL = np.empty([0,len(freq)])

# Loop through all the scans in the "scan_num" list:
for i in scan_id:
    file = h5py.File(dir_in_az+'dva_survey_raw_scan_'+f"{i:04}"+'.h5','r')
    print(i,file)
    RR,LL,reRL,imRL,trim_flag,dec,ra,el,az,t,noise = concatenate_data(file,RR,LL,reRL,imRL,trim_flag,dec,
                                                                      ra,el,az,t,noise,trim_az_exist)
    
t_set_plt = Time(t, format='isot',scale='utc').mjd

In [ ]:
mask_idx = []
with open('../DVA2/DATA/PersistRFI_v01_Jul13.txt') as fp:
    for line in fp:
        if line.split()[0] != 'index':
            mask_idx.append(line.split()[0])
mask_idx = np.array(mask_idx).astype(int)

In [ ]:
reRL_masked = reRL.copy()
imRL_masked = imRL.copy()
RR_masked = RR.copy()
LL_masked = LL.copy()

reRL_masked[:,mask_idx] = np.nan
imRL_masked[:,mask_idx] = np.nan
RR_masked[:,mask_idx] = np.nan
LL_masked[:,mask_idx] = np.nan

In [ ]:
reRL_masked[noise == 1] = np.nan
imRL_masked[noise == 1] = np.nan

In [ ]:
RR_masked[noise == 1] = np.nan
LL_masked[noise == 1] = np.nan

In [ ]:
reRL_masked_avg = reRL_masked.copy()
imRL_masked_avg = imRL_masked.copy()

for i in range(0,len(scan_id)):
    print(i)
    w = np.where((t_set_plt>=scan_start_mjd[i]) & (t_set_plt<=scan_stop_mjd[i]))[0]
    
    reRL_masked_avg[w,:] = np.nanmedian(reRL_masked[w,:],axis=0)
    imRL_masked_avg[w,:] = np.nanmedian(imRL_masked[w,:],axis=0)

In [ ]:
%%time

fplot = 800
wf = np.where(abs(freq-fplot)<df/2)[0][0]
print(freq[wf])

fig1, axs = plt.subplots(2,1,figsize=(20,14))
fs = 16

for i in range(0,len(scan_id)):
    w = np.where((t_set_plt>=scan_start_mjd[i]) & (t_set_plt<=scan_stop_mjd[i]))[0]
    extent = [scan_start_mjd[i],scan_stop_mjd[i],freq[0],freq[-1]]
    
    im1 = axs[0].imshow(reRL_masked[w,:].T,aspect='auto',vmin=-1e6,vmax=1e6,origin='lower',
                        extent=extent,cmap='RdBu_r')
    im11 = axs[1].imshow(reRL_masked_avg[w,:].T,aspect='auto',vmin=-1e6,vmax=1e6,origin='lower',
                        extent=extent,cmap='RdBu_r')   
    
cbar1= fig1.colorbar(im1,ax=axs[0])
cbar1.ax.tick_params(labelsize=fs) 
cbar1.set_label('Power (dB)', fontsize=fs)

cbar11= fig1.colorbar(im11,ax=axs[1])
cbar11.ax.tick_params(labelsize=fs) 
cbar11.set_label('Power (dB)', fontsize=fs)

for i in range(0,2):
    axs[i].set_xlim(t_set_plt[0],t_set_plt[-1])
    axs[i].tick_params(axis='both', labelsize=fs,labelbottom=True)
    axs[i].xaxis.set_major_formatter(mdates.DateFormatter('%H:%M:%S'))
    axs[i].fmt_xdata = mdates.DateFormatter('%H:%M:%S')
    axs[i].set_xlabel('Time (UTC)',fontsize=fs)        
    axs[i].set_ylim(freq[0],freq[-1])
    axs[i].set_ylabel('Frequency (MHz)',fontsize=fs)
    
#plt.tight_layout()
#plt.savefig('../DVA_PLOTS/gains_waterfall_phase1_day'+day+'.jpg')

In [ ]:
%%time

fplot = 800
wf = np.where(abs(freq-fplot)<df/2)[0][0]
print(freq[wf])

fig1, axs = plt.subplots(2,1,figsize=(20,16))
fs = 16

for i in range(0,len(scan_id)):
    w = np.where((t_set_plt>=scan_start_mjd[i]) & (t_set_plt<=scan_stop_mjd[i]))[0]
    extent = [scan_start_mjd[i],scan_stop_mjd[i],freq[0],freq[-1]]
  
    im2 = axs[2].imshow(imRL_masked[w,:].T,aspect='auto',vmin=-1e6,vmax=1e6,origin='lower',
                        extent=extent,cmap='RdBu_r')
    im22 = axs[3].imshow(imRL_masked_avg[w,:].T,aspect='auto',vmin=-1e6,vmax=1e6,origin='lower',
                        extent=extent,cmap='RdBu_r')

cbar2 = fig1.colorbar(im2,ax=axs[2])
cbar2.ax.tick_params(labelsize=fs) 
cbar2.set_label('Gain', fontsize=fs)

cbar22 = fig1.colorbar(im22,ax=axs[3])
cbar22.ax.tick_params(labelsize=fs) 
cbar22.set_label('Gain', fontsize=fs)

for i in range(0,2):
    axs[i].set_xlim(t_set_plt[0],t_set_plt[-1])
    axs[i].tick_params(axis='both', labelsize=fs,labelbottom=True)
    axs[i].xaxis.set_major_formatter(mdates.DateFormatter('%H:%M:%S'))
    axs[i].fmt_xdata = mdates.DateFormatter('%H:%M:%S')
    axs[i].set_xlabel('Time (UTC)',fontsize=fs)        
    axs[i].set_ylim(freq[0],freq[-1])
    axs[i].set_ylabel('Frequency (MHz)',fontsize=fs)
    
#plt.tight_layout()
#plt.savefig('../DVA_PLOTS/gains_waterfall_phase1_day'+day+'.jpg')

In [ ]:
%%time

fig1, axs = plt.subplots(3,1,figsize=(20,20))
fs = 12

for i in range(0,len(scan_id)):
    w = np.where((t_set_plt>=scan_start_mjd[i]) & (t_set_plt<=scan_stop_mjd[i]))[0]
    extent = [scan_start_mjd[i],scan_stop_mjd[i],freq[0],freq[-1]]

    im0 = axs[0].imshow(10*np.log10(LL_masked[w,:]).T,aspect='auto',vmin=69,vmax=75,origin='lower',
                        extent=extent,cmap='viridis')
    axs[0].set_title('LL', fontsize=fs)
    im1 = axs[1].imshow(reRL_masked[w,:].T,aspect='auto',vmin=-1e6,vmax=1e6,origin='lower',
                        extent=extent,cmap='RdBu_r')
    axs[1].set_title('re(RL)', fontsize=fs)
    im2 = axs[2].imshow(imRL_masked[w,:].T,aspect='auto',vmin=-1e6,vmax=1e6,origin='lower',
                        extent=extent,cmap='RdBu_r')  
    axs[2].set_title('im(RL)', fontsize=fs)

cbar0= fig1.colorbar(im0,ax=axs[0])
cbar0.ax.tick_params(labelsize=fs) 
cbar0.set_label('Power (dB)', fontsize=fs)    
    
cbar1= fig1.colorbar(im1,ax=axs[1])
cbar1.ax.tick_params(labelsize=fs) 
cbar1.set_label('Power (spectrometer units)', fontsize=fs)

cbar2= fig1.colorbar(im2,ax=axs[2])
cbar2.ax.tick_params(labelsize=fs) 
cbar2.set_label('Power (spectrometer units)', fontsize=fs)

for i in range(0,3):
    axs[i].set_xlim(t_set_plt[0],t_set_plt[-1])
    axs[i].tick_params(axis='both', labelsize=fs,labelbottom=True)
    axs[i].xaxis.set_major_formatter(mdates.DateFormatter('%H:%M:%S'))
    axs[i].fmt_xdata = mdates.DateFormatter('%H:%M:%S')
    axs[i].set_xlabel('Time (UTC)',fontsize=fs)        
    axs[i].set_ylim(freq[0],freq[-1])
    axs[i].set_ylabel('Frequency (MHz)',fontsize=fs)
    
#plt.tight_layout()
plt.savefig('../DVA2/PLOTS/pol_waterfall_phase1_day_v2'+day+'.pdf')

In [ ]:
fig1, axs = plt.subplots(3,1,figsize=(14,16))
fs = 12

scansplot = [0,4,8,12,16,20]

for i in scansplot:
    w = np.where((t_set_plt>=scan_start_mjd[i]) & (t_set_plt<=scan_stop_mjd[i]))[0]
    axs[0].scatter(freq,reRL_masked_avg[w[0],:],alpha=0.7,s=0.2,label='scan '+str(i+1))
    axs[0].set_ylim(-3.5e6,3.5e6)
    axs[0].set_title('Median of re(RL) over azimuth scans for survey day '+str(day))
    
    axs[1].scatter(freq,imRL_masked_avg[w[0],:],alpha=0.7,s=0.2,label='scan '+str(i+1))
    axs[1].set_ylim(-3.5e6,3.5e6)
    axs[1].set_title('Median of im(RL) over azimuth scans for survey day '+str(day))
    
    #axs[2].plot(freq,10*np.log10(RR[100,:]))
    #axs[2].plot(freq,10*np.log10(RR_masked[100,:]))
    
    axs[2].plot(freq,np.sqrt(reRL[100,:]**2+imRL[100,:]**2))
    axs[2].plot(freq,np.sqrt(reRL_masked[100,:]**2+imRL_masked[100,:]**2))
    axs[2].set_ylim(0,2e7)
    axs[2].set_title('Polarized intensity spectrum and RFI mask')
    axs[2].set_xlabel('Frequency (MHz)',fontsize=fs)
    
    for j in range(0,3):
        axs[j].set_xlim(350,1030)
        axs[j].grid(True)
        axs[j].legend(markerscale=10, fontsize=fs)
        axs[j].set_ylabel('Power (raw spectrometer units)',fontsize=fs)
        
plt.savefig('../DVA2/PLOTS/pol_median_phase1_day'+day+'.jpg')